## 1. Set the project and import Charcrete database

In [109]:
# === Switch to kernel bw25! ===

# Brightway libraries
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as ba
import bw_processing as bwp

# General libraries
import numpy as np
import pandas as pd
import math
from fs.zipfs import ZipFS
import json                        # Library for working with json files
from pathlib import Path           # Library for working with paths in different OS     
import matplotlib.pyplot as plt    # Library for creating plots
import os

# Visualization
import plotly.graph_objects as go

# Uncertainties
import stats_arrays as sa

In [2]:
bd.projects.set_current("ei38-teaching-25")

## 2. Importing Charcrete database

In [3]:
#finding the relative path
filepath = os.getcwd()+"/data/Charcrete_database_38.xlsx"

In [5]:
if "Charcrete_38" in bd.databases:
    print("Charcrete_38 database already exists")
else:
    # 1. Specify filepath to your foreground inventories.
    charcrete_path = filepath
    # 2. Create an instance of a class that contains basic methods for importing a database from an excel file.
    charcrete = bi.ExcelImporter(charcrete_path)  
    # 3. `apply_strategies` is one of such basic methods, it makes sure units, locations, etc are in correct format.
    charcrete.apply_strategies()
    # 4. Next step is to link your foreground exchanges to existing databases by matching relevant exchanges fields.
    charcrete.match_database("biosphere3", fields=("name", "unit", "categories"))
    charcrete.match_database("ei 3.8 cutoff", fields=("name", "location", "unit"))
    #charcrete.metadata.pop(None)  # Remove metadata None entry. TODO
    # 5. If everything is linked, write database so that it is saved in your project.
    if charcrete.all_linked:
        charcrete.write_database()

Extracted 1 worksheets in 0.01 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.79 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
5 datasets
23 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 da

In [6]:
bd.databases

Databases dictionary with 7 object(s):
	Charcrete_38
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	Korean Motors Chaebol
	biosphere3
	ei 3.8 cutoff
	강한 국가를 위한 영광스러운 구리

## 3. Check LCA on charcrete to ensure everything is working fine

In [8]:
char_db = bd.Database("Charcrete_38")
ei_db = bd.Database("ei 3.8 cutoff")

In [12]:
char_p = [act for act in char_db if "Charcrete" in act['name']][0]

In [13]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')
method = bd.Method(ipcc)

In [14]:
def calculateLCA(act, amount, method):
    fu, data_objs, _ = bd.prepare_lca_inputs({act: amount}, method=ipcc)
    lca = bc.LCA(fu, data_objs=data_objs)
    lca.lci()
    lca.lcia()
    return lca.score

In [15]:
calculateLCA(char_p, 1, ipcc)

317.2283827379036

## 4.WiP on correlated samples

In [18]:
[act for act in ei_db if 'concrete' in act['name']][0].as_dict()

{'comment': 'Hardening is assumed to be by air-drying. It is assumed that there is wastewater and solid waste, but as no exact description on the composition of the wastewater and solid waste is available a proxy for each ("treatment, concrete production effluent, to wastewater treatment, class 3" and "disposal, municipal solid waste, 22.9% water, to municipal incineration") is used. The lifespan of the plant is assumed to be 50 years.\nLightweight concrete block. Image source: http://keramzit.by/data/pictures/poduct/block1.png\nImage: https://db3.ecoinvent.org/images/0b755024-3429-4acb-a241-cffdc608c3ff\nGeography:  For some exchanges CH, RER and GLO modules have been used as proxy.\nTechnology:  The raw material mixture (aggregates, cement and water) of lightweight concrete blocks are poured into a mould then squeezed and finaly hardened by air (assumption for this study)',
 'classifications': [('EcoSpold01Categories',
   'construction materials/concrete'),
  ('ISIC rev.4 ecoinvent',

In [19]:
concrete_ei_20MPa = [act for act in ei_db if "concrete production 20MPa" in act["name"]]
concrete_ei_20MPa

['concrete production 20MPa' (cubic meter, CA-QC, None),
 'concrete production 20MPa' (cubic meter, ZA, None),
 'concrete production 20MPa' (cubic meter, RoW, None),
 'concrete production 20MPa' (cubic meter, North America without Quebec, None)]

In [37]:
exc_list = []
for act in concrete_ei_20MPa :
     for exc in act.technosphere():
         if exc['unit']== 'kilogram': 
            exc_list.append(exc['name'])
         else: 
            pass
exc_list = set(exc_list)
exc_list

{'acetic acid, without water, in 98% solution state',
 'alkylbenzene sulfonate, linear, petrochemical',
 'cement, Portland',
 'cement, portland fly ash cement 21-35%',
 'chemical, organic',
 'ethylene oxide',
 'gravel, round',
 'lubricating oil',
 'sand',
 'steel, low-alloyed, hot rolled',
 'synthetic rubber',
 'tap water',
 'waste concrete'}

In [54]:
len(concrete_ei_20MPa)

4

In [47]:
density_mapping = []
for act in concrete_ei_20MPa :
    d=0
    for exc in act.technosphere():
        if exc['unit']== 'kilogram': 
            d = d + exc['amount']     
        else: 
            pass
    density_mapping.append([act['name'],d])
density_mapping

[['concrete production 20MPa', 2330.772867449367],
 ['concrete production 20MPa', 2415.9284602953603],
 ['concrete production 20MPa', 2415.9284602953594],
 ['concrete production 20MPa', 2338.1568988804133]]

In [73]:
def ComputeDensity(act_list_in_cubic_meter):
    density_mapping = []
    for act in act_list_in_cubic_meter :
        d=0
        try : 
            act['unit']=='cubic meter'
        except ValueError:
            raise ValutError("activity unit has to be cubic meter")
        for exc in act.technosphere():
            if exc['unit']== 'kilogram': 
                d = d + exc['amount']
            else:
                pass
        density_mapping.append([act['name'], act['location'],d])
    return pd.DataFrame(density_mapping , columns = ["Activity Name", "Location","density"])

In [74]:
ComputeDensity(concrete_ei_20MPa)

,Activity Name,Location,density
0,concrete production 20MPa,CA-QC,2330.772867
1,concrete production 20MPa,ZA,2415.928460
2,concrete production 20MPa,RoW,2415.928460
3,concrete production 20MPa,North America without Quebec,2338.156899


In [105]:
l = [
act for act in ei_db 
    if 'concrete' in act['name'] 
    and act['unit']=='cubic meter' 
    and 'market' not in act['name']
    and 'treatment' not in act['name']
    and 'slab' not in act['name']
]

In [106]:
density_table = ComputeDensity(l)

In [107]:
density_table.sort_values(by = 'density')

,Activity Name,Location,density
64,"concrete production, 40MPa, ready-mix, with ce...",RoW,2182.69546
21,"concrete production, 40MPa, ready-mix, with ce...",BR,2182.69546
59,"lean concrete production, with cement CEM II/B",CH,2244.80000
9,"lean concrete production, with cement CEM II/A",CH,2244.80000
68,"lean concrete production, with cement CEM II/A",RoW,2244.80000
...,...,...,...
55,"concrete production, 40MPa, ready-mix, with ce...",PE,2514.71400
49,"concrete production, 40MPa, ready-mix, with ce...",RoW,2514.71400
67,"concrete production, 40MPa, ready-mix, with ce...",RoW,2514.71400
60,"concrete production, 20MPa, ready-mix, with ce...",PE,2552.71400


In [91]:
for i in range (0:len(density_table))
    if density_table.loc(i)['density']

102

In [ ]:
dp = bwp.create_datapackage()